In [ ]:
# | default_exp workflow
%load_ext autoreload
# automatically reload (local) python modules if they are updated
%autoreload 2

In [ ]:
# | hide
from nbdev.showdoc import *

# Workflow

    Define workflow for automatically updating, training and deploying your ML model!


***input:*** data_etl, model class and evaluation notebooks

***output:*** script for executing the ML model update workflow

***description:***

A ML model update workflow allows you to automatically reload your data, train, evaluate and save your ML model to a model store.

Note that by following the notebook templates you have already done most of the work - the notebooks **are** the workflow!

So, in this notebook you define a script to automatically execute the other notebooks with the [papermill](https://papermill.readthedocs.io/) tool. Note, that you can input parameters to the notebooks!

Edit this and other text cells to describe your project. 

Remember that you can utilize the `# | export` tag to export cell commands to `your_module_name/workflow.py`.

## Import relevant modules

In [ ]:
from datetime import datetime
import papermill
from pathlib import Path
import os

# your code here

## Define notebook parameters

In [ ]:
# Tag this cell as 'parameters' to utilise papermills notebook parameterization!
seed = 0
setup = "setup_1a"
# your code here

Make direct derivations from the paramerters:

In [ ]:
# your code here

## Define workflow

In [ ]:
"""
A workflow to re-run your machine learning workflow automatically.

This example script will
- rebuild your python module
- run data_etl notebook to reload and clean data
- run model_class notebook to sw test your model
- run train_test_val notebook to train and evaluate your model with full data,
    and save it for further use

Feel free to edit!
"""

# create version name for model and backup folder to save notebooks to after running them

# WARNING: make sure to commit all changes before versioning a model,
# to be able to match the model to source code version!
model_timestamp = datetime.now()
time = (
    str(model_timestamp)
    .replace("-", "")
    .replace(" ", "_")
    .replace(":", "")
    .replace(".", "_")
)
branch = os.popen("git symbolic-ref -q --short HEAD").read().strip()
head = os.popen("git rev-parse --short HEAD").read().strip()

model_version = "_".join([time, branch, head, setup])

# create backup folder if it does not exist
cwd = Path().cwd()
backup_path = cwd.parent / "local_data" / "ml_pipe_notebook_backups" / model_version

try:
    backup_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:  # do not overwrite
    pass

# make sure changes are updated to module
os.system("nbdev_export")

# run workflow
for notebook in [
    "00_data_etl.ipynb",
    "01_model_class.ipynb",
    "02_train_test_validate.ipynb",
]:
    papermill.execute_notebook(
        notebook,  # this notebook will be executed
        backup_path
        / ("_" + notebook),  # this is where the executed notebook will be saved
        # (notebooks named with '_' -prefix are ignored by nbdev!)
        parameters={
            "seed": 1,
            "model_version": model_version,
        },  # you can change notebook parameters
    )